In [1]:
!git clone https://github.com/wandb/edu.git
%cd edu/rag-advanced
!pip install -qqq -r requirements.txt
%cd notebooks

import nltk

nltk.download("wordnet")

Cloning into 'edu'...
remote: Enumerating objects: 4510, done.
remote: Counting objects: 100% (1393/1393), done.
remote: Compressing objects: 100% (531/531), done.
remote: Total 4510 (delta 1058), reused 1024 (delta 859), pack-reused 3117 (from 1)
Receiving objects: 100% (4510/4510), 28.01 MiB | 6.17 MiB/s, done.
Resolving deltas: 100% (2472/2472), done.
Updating files: 100% (461/461), done.
/content/edu/rag-advanced
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:0

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import getpass
import os

import wandb

os.environ["COHERE_API_KEY"] = getpass.getpass("Please enter your COHERE_API_KEY")
wandb.login()

Please enter your COHERE_API_KEY··········


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import pathlib
from typing import List

import weave

from scripts.utils import display_source

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
WANDB_PROJECT = "rag-course"

run = wandb.init(
    project=WANDB_PROJECT,
    group="Chapter 1",
)

wandb: Currently logged in as: shikhar_dave (shikhar_dave-iit-jodhpur). Use `wandb login --relogin` to force relogin


In [6]:
documents_artifact = run.use_artifact(
    f"rag-course/dev/wandb_docs:latest", type="dataset"
)
data_dir = "../data/wandb_docs"

docs_dir = documents_artifact.download(data_dir)

wandb:   406 of 406 files downloaded.  


In [7]:
docs_dir = pathlib.Path(docs_dir)
docs_files = sorted(docs_dir.rglob("*.md"))

print(f"Number of files: {len(docs_files)}\n")
print("First 5 files:\n{files}".format(files="\n".join(map(str, docs_files[:5]))))

Number of files: 406

First 5 files:
../data/wandb_docs/guides/app/features/anon.md
../data/wandb_docs/guides/app/features/custom-charts/intro.md
../data/wandb_docs/guides/app/features/custom-charts/walkthrough.md
../data/wandb_docs/guides/app/features/intro.md
../data/wandb_docs/guides/app/features/notes.md


In [8]:
print(docs_files[0].read_text())

---
description: Log and visualize data without a W&B account
displayed_sidebar: default
---

# Anonymous Mode

Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.

Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`

:::info
**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.
:::

### How does someone without an account see results?

If someone runs your script and you have to set `anonymous="allow"`:

1. **Auto-create temporary account:** W&B checks for an account that's already signed in. If there's no account, we automatically create a new anonymous account and save that API key for the session.
2. **Log results quickly:** T

In [9]:
# We'll store the files as dictionaries with some content and metadata
data = []
for file in docs_files:
    content = file.read_text()
    data.append(
        {
            "content": content,
            "metadata": {
                "source": str(file.relative_to(docs_dir)),
                "raw_tokens": len(content.split()),
            },
        }
    )
data[:2]

[{'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the ses

In [10]:
total_tokens = sum(map(lambda x: x["metadata"]["raw_tokens"], data))
print(f"Total Tokens in dataset: {total_tokens}")

Total Tokens in dataset: 260530


In [11]:
WANDB_PROJECT = "rag-course"
weave_client = weave.init(WANDB_PROJECT)

Logged in as Weights & Biases user: shikhar_dave.
View Weave data at https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/weave


In [12]:
# build weave dataset
raw_data = weave.Dataset(name="raw_data", rows=data)

# publish the dataset
weave.publish(raw_data)

📦 Published to https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/weave/objects/raw_data/versions/FtXM1nny6EREQTDUYbI7olXelWIWwvUwlVXYNhXYsPM


ObjectRef(entity='shikhar_dave-iit-jodhpur', project='rag-course', name='raw_data', digest='FtXM1nny6EREQTDUYbI7olXelWIWwvUwlVXYNhXYsPM', extra=())

In [13]:
# These are hyperparameters of our ingestion pipeline

CHUNK_SIZE = 300
CHUNK_OVERLAP = 0


def split_into_chunks(
    text: str, chunk_size: int = CHUNK_SIZE, chunk_overlap: int = CHUNK_OVERLAP
) -> List[str]:
    """Function to split the text into chunks of a maximum number of tokens
    ensure that the chunks are of size CHUNK_SIZE and overlap by chunk_overlap tokens
    use the `tokenizer.encode` method to tokenize the text
    """
    tokens = text.split()
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk = tokens[start:end]
        chunks.append(" ".join(chunk))
        start = end - chunk_overlap
    return chunks

In [14]:
# download the `raw_data` Dataset
raw_data = weave.ref(
    "weave:///rag-course/rag-course/object/raw_data:nuZosGsP58MXKxhluN2hzvKK9XB8xSxlTuOBmEzWzLo"
).get()
raw_data = weave.ref("raw_data:v0").get()

# this is how we index into the data
print(raw_data.rows[:2])

[WeaveDict({'content': '---\ndescription: Log and visualize data without a W&B account\ndisplayed_sidebar: default\n---\n\n# Anonymous Mode\n\nAre you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first.\n\nAllow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)`\n\n:::info\n**Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n:::\n\n### How does someone without an account see results?\n\nIf someone runs your script and you have to set `anonymous="allow"`:\n\n1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key f

In [15]:
chunked_data = []
for doc in raw_data.rows:
    chunks = split_into_chunks(doc["content"])
    for chunk in chunks:
        chunked_data.append(
            {
                "content": chunk,
                "metadata": {
                    "source": doc["metadata"]["source"],
                    "raw_tokens": len(chunk.split()),
                },
            }
        )

print(chunked_data[:2])

[{'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log results qui

In [16]:
from scripts.preprocess import make_text_tokenization_safe
display_source(make_text_tokenization_safe)

   1 def make_text_tokenization_safe(                                                                              
   2     content: str, special_tokens_set: set = get_special_tokens_set()                                          
   3 ) -> str:                                                                                                     
   4     """                                                                                                       
   5     Makes the text safe for tokenization by removing special tokens.                                          
   6                                                                                                               
   7     Args:                                                                                                     
   8         content: A string containing the text to be processed.                                                
   9         special_tokens_set: A set of special tokens to be removed from the text.                              
  10                                                                                                               
  11     Returns:                                                                                                  
  12         A string with the special tokens removed.                                                             
  13     """                                                                                                       
  14                                                                                                               
  15     def remove_special_tokens(text: str) -> str:                                                              
  16         """                                                                                                   
  17         Removes special tokens from the given text.                                                           
  18                                                                                                               
  19         Args:                                                                                                 
  20             text: A string representing the text.                                                             
  21                                                                                                               
  22         Returns:                                                                                              
  23             The text with special tokens removed.                                                             
  24         """                                                                                                   
  25         for token in special_tokens_set:                                                                      
  26             text = text.replace(token, "")                                                                    
  27         return text                                                                                           
  28                                                                                                               
  29     cleaned_content = remove_special_tokens(content)                                                          
  30     return cleaned_content                                                                                    
  31                                                                                                               

In [17]:
cleaned_data = []
for doc in chunked_data:
    cleaned_doc = doc.copy()
    cleaned_doc["cleaned_content"] = make_text_tokenization_safe(doc["content"])
    cleaned_doc["metadata"]["cleaned_tokens"] = len(
        cleaned_doc["cleaned_content"].split()
    )
    cleaned_data.append(cleaned_doc)
print(cleaned_data[:2])

[{'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log results qui

In [18]:
dataset = weave.Dataset(name="chunked_data", rows=cleaned_data)
weave.publish(dataset)

📦 Published to https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/weave/objects/chunked_data/versions/2QCiHXRTg3HGY1QYBLC7yGWar7TYVmi6YulXCOm7Fp4


ObjectRef(entity='shikhar_dave-iit-jodhpur', project='rag-course', name='chunked_data', digest='2QCiHXRTg3HGY1QYBLC7yGWar7TYVmi6YulXCOm7Fp4', extra=())

In [19]:
chunked_data = weave.ref(
    "weave:///rag-course/rag-course/object/chunked_data:Lt6M8qCUICD1JZTlMYzuDLTVtvFYESxvj3tcAIoPrtE"
).get()
chunked_data = weave.ref("chunked_data:v0").get()
print(chunked_data.rows[:2])

[WeaveDict({'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log r

In [20]:
from scripts.retriever import TFIDFRetriever
display_source(TFIDFRetriever)

   1 class TFIDFRetriever(weave.Model):                                                                            
   2     """                                                                                                       
   3     A retriever model that uses TF-IDF for indexing and searching documents.                                  
   4                                                                                                               
   5     Attributes:                                                                                               
   6         vectorizer (TfidfVectorizer): The TF-IDF vectorizer.                                                  
   7         index (list): The indexed data.                                                                       
   8         data (list): The data to be indexed.                                                                  
   9     """                                                                                                       
  10                                                                                                               
  11     vectorizer: TfidfVectorizer = TfidfVectorizer()                                                           
  12     index: list = None                                                                                        
  13     data: list = None                                                                                         
  14                                                                                                               
  15     def index_data(self, data):                                                                               
  16         """                                                                                                   
  17         Indexes the provided data using TF-IDF.                                                               
  18                                                                                                               
  19         Args:                                                                                                 
  20             data (list): A list of documents to be indexed. Each document should be a dictionary              
  21                          containing a key 'cleaned_content' with the text to be indexed.                      
  22         """                                                                                                   
  23         self.data = data                                                                                      
  24         docs = [doc["cleaned_content"] for doc in data]                                                       
  25         self.index = self.vectorizer.fit_transform(docs)                                                      
  26                                                                                                               
  27     @weave.op()                                                                                               
  28     def search(self, query, k=5):                                                                             
  29         """                                                                                                   
  30         Searches the indexed data for the given query using cosine similarity.                                
  31                                                                                                               
  32         Args:                                                                                                 
  33             query (str): The search query.                                                                    
  34             k (int): The number of top results to return. Default is 5.                                       
  35                                                    

In [21]:
retriever = TFIDFRetriever()
retriever.index_data(list(map(dict, chunked_data.rows)))

In [22]:
query = "How do I use W&B to log metrics in my training script?"
search_results = retriever.search(query)
for result in search_results:
    print(result)

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/01926cdb-09b1-7683-b884-b942050a0769
{'source': 'guides/technical-faq/general.md', 'text': '--- displayed_sidebar: default --- # General ### What does `wandb.init` do to my training process? When `wandb.init()` is called from your training script an API call is made to create a run object on our servers. A new process is started to stream and collect metrics, thereby keeping all threads and logic out of your primary process. Your script runs normally and writes to local files, while the separate process streams them to our servers along with system metrics. You can always turn off streaming by running `wandb off` from your training directory, or setting the `WANDB_MODE` environment variable to `offline`. ### Does your tool track or store training data? You can pass a SHA or other unique identifier to `wandb.config.update(...)` to associate a dataset with a training run. W&B does not store any data unless `wandb.save` is call

In [23]:
from scripts.response_generator import SimpleResponseGenerator
display_source(SimpleResponseGenerator)

   1 class SimpleResponseGenerator(weave.Model):                                                                   
   2     """                                                                                                       
   3     A simple response generator model using Cohere's API.                                                     
   4                                                                                                               
   5     Attributes:                                                                                               
   6         model (str): The model name to be used for generating responses.                                      
   7         prompt (str): The prompt to be used for generating responses.                                         
   8         client (cohere.ClientV2): The Cohere client for interacting with the Cohere API.                      
   9     """                                                                                                       
  10                                                                                                               
  11     model: str                                                                                                
  12     prompt: str                                                                                               
  13     client: cohere.ClientV2 = None                                                                            
  14                                                                                                               
  15     def __init__(self, **kwargs):                                                                             
  16         """                                                                                                   
  17         Initialize the SimpleResponseGenerator with the provided keyword arguments.                           
  18         Sets up the Cohere client using the API key from environment variables.                               
  19         """                                                                                                   
  20         super().__init__(**kwargs)                                                                            
  21         self.client = cohere.ClientV2(                                                                        
  22             api_key=os.environ["COHERE_API_KEY"],                                                             
  23         )                                                                                                     
  24                                                                                                               
  25     @weave.op()                                                                                               
  26     def generate_context(self, context: List[Dict[str, any]]) -> List[Dict[str, any]]:                        
  27         """                                                                                                   
  28         Generate a list of contexts from the provided context list.                                           
  29                                                                                                               
  30         Args:                                                                                                 
  31             context (List[Dict[str, any]]): A list of dictionaries containing context data.                   
  32                                                                                                               
  33         Returns:                                                                                              
  34             List[Dict[str, any]]: A list of dictionaries with 'source' and 'text' keys.                       
  35         """                                        

In [24]:
INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
print(INITIAL_PROMPT)

Answer to the following question about W&B. Provide an helful and complete answer based only on the provided documents.



In [25]:
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
answer = response_generator.generate_response(query, search_results)
print(answer)

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/01926ce5-cfa3-71b3-b85a-ce04130d5b89
You can use the W&B to log metrics in your training script by following these steps:
1. Call `wandb.init()` in your training script. This creates a run object on W&B servers and starts a new process to stream and collect metrics.
2. Your script runs normally and writes to local files, while the separate process streams them to W&B servers along with system metrics. 
3. Log metrics by calling `wandb.log()` at relevant points in your script. For example, you might call this every time the accuracy changes. You can log images, videos, and other media types, as well as scalars.
4. To finish your W&B run, call `wandb.finish()` when your training script ends.


In [26]:
from scripts.rag_pipeline import SimpleRAGPipeline

display_source(SimpleRAGPipeline)

   1 class SimpleRAGPipeline(weave.Model):                                                                         
   2     """                                                                                                       
   3     A simple RAG (Retrieval-Augmented Generation) pipeline.                                                   
   4                                                                                                               
   5     Attributes:                                                                                               
   6         retriever (weave.Model): The model used for retrieving relevant documents.                            
   7         response_generator (weave.Model): The model used for generating responses.                            
   8         top_k (int): The number of top documents to retrieve.                                                 
   9     """                                                                                                       
  10                                                                                                               
  11     retriever: weave.Model = None                                                                             
  12     response_generator: weave.Model = None                                                                    
  13     top_k: int = 5                                                                                            
  14                                                                                                               
  15     @weave.op()                                                                                               
  16     def predict(self, query: str):                                                                            
  17         """                                                                                                   
  18         Predicts a response based on the input query.                                                         
  19                                                                                                               
  20         Args:                                                                                                 
  21             query (str): The input query string.                                                              
  22                                                                                                               
  23         Returns:                                                                                              
  24             The generated response based on the retrieved context.                                            
  25         """                                                                                                   
  26         context = self.retriever.predict(query, self.top_k)                                                   
  27         return self.response_generator.predict(query, context)                                                
  28                                                                                                               

In [27]:
# Initialize the response generator
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)

# Bring them together as a RAG pipeline
rag_pipeline = SimpleRAGPipeline(
    retriever=retriever, response_generator=response_generator, top_k=5
)

In [28]:
response = rag_pipeline.predict("How do I get get started with wandb?")
print(response, sep="\n")

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/01926ce7-c0ff-7311-b65b-b441b1c60ba0
To get started with wandb.init(), you should first call it from your training script – this will create a run object on W&B servers. A new process is then started to stream and collect metrics, keeping all threads and logic out of your primary process. 

Your script runs as normal, but now writes to local files, while the separate process streams them to the servers along with system metrics. You can turn off streaming by running wandb off from your training directory, or setting the WANDB_MODE environment variable to offline.

You can also use wandb.login() in your Jupyter notebook to get started with W&B. After running this command, start a new cell with %%wandb to see live graphs.
